In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os
from io import StringIO
from collections import OrderedDict

os.chdir('D:\Cloud\Dropbox\postdoc\summa\summaData\columbia10')

In [16]:
def strip_line(s):
    s = s.strip()
    if s.startswith("'"):
        istart = 1
        iend = s.find("'", 1)
    else:
        istart = 0
        iend = s.find(' ') + 1
    return s[istart:iend]

def read_file_manager(f_man):
    with open(f_man, "r") as f:
        lines = f.readlines()
        lines = [strip_line(l) for l in lines if l[0] != '!']
        i = 0
        keys = ['fman_ver', 'setting_path', 'input_path', 'output_path', 'decision', 
                'meta_time', 'meta_attr', 'meta_type', 'meta_force', 'meta_localpar',
                'ouput_file', 'meta_index', 'meta_basinpar', 'meta_basinvar',
                'local_attr', 'local_par', 'basin_par', 'forcing_list', 'initial_cond',
                'para_trial', 'output_prefix']
        file_man = OrderedDict(zip(keys, lines))
        
        return file_man

    
fman = 'settings_org/syntheticTestCases/wigmosta1999/summa_fileManager-exp1.txt'
#%time file_man = read_file_manager(fman)

In [9]:
np.version.release

True

In [3]:
def write_file_manager(file_man, file_name):    
    # check the maximum with for the first column
    lens = [len(v) for v in file_man.values()]
    width_max = max(lens) + 20
    with open(file_name, "w") as f:
        for k in file_man:
            line = "'" + file_man[k] + "'"
            line = line.ljust(width_max)
            line = line + '! ' + k + '\n'
            f.write(line)
        f.close()
#%time write_file_manager(file_man, file_name)

'settings_org/syntheticTestCases/wigmosta1999/summa_fileManager-exp1.txt1'

In [16]:
lens = [len(v) for v in file_man.values()]
max(lens)

67

In [83]:
# find all inital state files
def search_ini_files():
    os.chdir('/home/mgou/Dropbox/postdoc/summa/summaTestCases')
    ini_files = []
    for dirpath, dirnames, filenames in os.walk('settings_org'):
        for f in filenames:
            if 'summa_zInitialCond' in f:
                ini_files.append(os.path.join(dirpath,f))
                os.listdir(dirpath)
%timeit search_ini_files()

1000 loops, best of 3: 1.56 ms per loop


In [82]:
# find all inital state files
def search_ini_files():
    os.chdir('/home/mgou/Dropbox/postdoc/summa/summaTestCases')
    ini_files = []
    for dirpath, dirnames, filenames in os.walk('settings_org'):
        for f in filenames:
            ini_files.append(os.path.join(dirpath,f)) if 'summa_zInitialCond' in str(f) else 'pass'
            
%timeit search_ini_files()

1000 loops, best of 3: 1.54 ms per loop


In [2]:
a = np.array([1,2,3])
a

In [14]:
ini_file = 'settings_org/' + file_man['initial_cond']
def read_ini_file(ini_file):
    scalar_var_line = ''
    layer_var_line = ''
    scalar_var_block = False
    layer_var_block = False
    with open(ini_file, "r") as f:
        lines = f.readlines()
        for l in lines:
            if l.startswith('!'):
                continue
            if 'start_scalar_icond' in l:
                scalar_var_block = True
                continue
            if 'end_scalar_icond' in l:
                scalar_var_block = False
                continue
            if 'start_layer_icond' in l:
                layer_var_block = True
                continue
            if 'end_layer_icond' in l:
                layer_var_block = False
                break
            if scalar_var_block:
                scalar_var_line += l
            if layer_var_block:
                layer_var_line += l
                
    scalar_var = pd.read_table(StringIO(scalar_var_line), 
                               sep=' ', 
                               skipinitialspace=True, 
                               header=None,
                               index_col=0,
                               squeeze=True)
    layer_var = pd.read_table(StringIO(layer_var_line), sep=' ', skipinitialspace=True)
    return (scalar_var, layer_var) 

#%time scalar_var, layer_var = read_ini_file(ini_file)

'syntheticTestCases/wigmosta1999/summa_zParamTrial-exp1.txt'

In [15]:
def check_hru_num(file):
    '''
    query the total number of hru in the model based on paramTrial
    '''
    with open(file, "r") as f:
            lines = f.readlines()
            lines = [l for l in lines if l[0] != '!']
            return (len(lines) - 1)
            
#%time check_hru_num('settings_org/' + file_man['para_trial'])        

In [17]:
def create_initi_nc(nhru, scalar_var, layer_var):
    nspec = 2
    nmidSoil = layer_var.layerType.value_counts()['soil']
    nifcSoil = nmidSoil + 1
    nmidToto = layer_var.shape[0]
    nifcToto = nmidToto + 1
    nscalarv = 1

    nSnow = np.repeat(nmidToto-nmidSoil, nhru).reshape([nscalarv, nhru]).astype(np.int32)
    nSoil = np.repeat(nmidSoil, nhru).reshape([nscalarv, nhru]).astype(np.int32)
    dt_init = np.repeat(scalar_var['dt_init'], nhru).reshape([nscalarv, nhru])
    scalarCanopyIce = np.repeat(scalar_var['scalarCanopyIce'], nhru).reshape([nscalarv, nhru])
    scalarCanopyLiq = np.repeat(scalar_var['scalarCanopyLiq'], nhru).reshape([nscalarv, nhru])
    scalarCanairTemp = np.repeat(scalar_var['scalarCanairTemp'], nhru).reshape([nscalarv, nhru])
    scalarCanopyTemp = np.repeat(scalar_var['scalarCanopyTemp'], nhru).reshape([nscalarv, nhru])
    scalarSnowAlbedo = np.repeat(scalar_var['scalarSnowAlbedo'], nhru).reshape([nscalarv, nhru])
    scalarSWE = np.repeat(scalar_var['scalarSWE'], nhru).reshape([nscalarv, nhru])
    scalarSnowDepth = np.repeat(scalar_var['scalarSnowDepth'], nhru).reshape([nscalarv, nhru])
    scalarSfcMeltPond = np.repeat(scalar_var['scalarSfcMeltPond'], nhru).reshape([nscalarv, nhru])
    scalarAquiferStorage = np.repeat(scalar_var['scalarAquiferStorage'], nhru).reshape([nscalarv, nhru])

    iLayerHeight_tmp = np.append(layer_var['iLayerHeight'], 
                                 layer_var.loc[nmidToto - 1,'iLayerHeight'] + layer_var.loc[nmidToto - 1,'mLayerDepth'])
    iLayerHeight = np.repeat(iLayerHeight_tmp, nhru).reshape([nifcToto, nhru])
    mLayerDepth  = np.repeat(layer_var['mLayerDepth'], nhru).reshape([nmidToto, nhru])
    mLayerTemp  = np.repeat(layer_var['mLayerTemp'], nhru).reshape([nmidToto, nhru])
    mLayerVolFracIce  = np.repeat(layer_var['mLayerVolFracIce'], nhru).reshape([nmidToto, nhru])
    mLayerVolFracLiq  = np.repeat(layer_var['mLayerVolFracLiq'], nhru).reshape([nmidToto, nhru])
    mLayerMatricHead  = np.repeat(layer_var['mLayerMatricHead'], nhru).reshape([nmidSoil, nhru])


    ini_nc = xr.Dataset({'nSnow':(['scalarv', 'hru'], nSnow),
                         'nSoil':(['scalarv', 'hru'], nSoil),
                         'dt_init':(['scalarv', 'hru'], dt_init),
                         'scalarCanopyIce':(['scalarv', 'hru'], scalarCanopyIce),
                         'scalarCanopyLiq':(['scalarv', 'hru'], scalarCanopyLiq),
                         'scalarCanairTemp':(['scalarv', 'hru'], scalarCanairTemp),
                         'scalarCanopyTemp':(['scalarv', 'hru'], scalarCanopyTemp),
                         'scalarSnowAlbedo':(['scalarv', 'hru'], scalarSnowAlbedo),
                         'scalarSWE':(['scalarv', 'hru'], scalarSWE),
                         'scalarSnowDepth':(['scalarv', 'hru'], scalarSnowDepth),
                         'scalarSfcMeltPond':(['scalarv', 'hru'], scalarSfcMeltPond),
                         'scalarAquiferStorage':(['scalarv', 'hru'], scalarAquiferStorage),
                         'iLayerHeight':(['ifcToto','hru'], iLayerHeight),
                         'mLayerDepth':(['midToto', 'hru'], mLayerDepth),
                         'mLayerTemp':(['midToto', 'hru'], mLayerTemp),
                         'mLayerVolFracIce':(['midToto', 'hru'], mLayerVolFracIce),
                         'mLayerVolFracLiq':(['midToto', 'hru'], mLayerVolFracLiq),
                         'mLayerMatricHead':(['midSoil', 'hru'], mLayerMatricHead)})

    return ini_nc

#nhru = check_hru_num('settings_org/' + file_man['para_trial'])      
#scalar_var, layer_var = read_ini_file('settings_org/' + file_man['initial_cond'])
#%time nc = create_initi_nc(nhru, scalar_var, layer_var)
#nc

In [18]:
### convert initial condition file to netcdf
def convert_ini_nc(root, file_man):
    nhru = check_hru_num(os.path.join(root, file_man['para_trial']))   
    scalar_var, layer_var = read_ini_file(os.path.join(root, file_man['initial_cond']))
    ini_nc = create_initi_nc(nhru, scalar_var, layer_var)
    # strip the initial condition file
    ini_nc_name = os.path.splitext(file_man['initial_cond'])[0] + '.nc'
    ini_nc.to_netcdf(os.path.join(root, ini_nc_name))
    return ini_nc_name
    

In [18]:
### convert forcing data to netcdf (done)
'summa_fileManager' in 'summa_fileManager_millerSand.txt'

True

In [10]:
### loop all the fileManager files
root = 'settings_org'
for dirpath, dirnames, filenames in os.walk(root):
    for filename in filenames:
        if 'summa_fileManager' in filename:
            print('converting ' + dirpath + '/' + filename)
            file_man = read_file_manager(os.path.join(dirpath, filename))
            # changing the output control file name
            file_man['ouput_file'] = 'meta/Model_Output.txt'
            # convert initial condition file to netcdf
            if not file_man['initial_cond'].endswith('.nc'):
                file_man['initial_cond'] = convert_ini_nc(root, file_man) 
            # write back the file manager
            write_file_manager(file_man, os.path.join(dirpath, filename))

converting settings_org\syntheticTestCases\celia1990/summa_fileManager_celia1990.txt
converting settings_org\syntheticTestCases\miller1998/summa_fileManager_millerClay.txt
converting settings_org\syntheticTestCases\miller1998/summa_fileManager_millerLoam.txt
converting settings_org\syntheticTestCases\miller1998/summa_fileManager_millerSand.txt
converting settings_org\syntheticTestCases\mizoguchi1990/summa_fileManager_mizoguchi.txt
converting settings_org\syntheticTestCases\wigmosta1999/summa_fileManager-exp1.txt
converting settings_org\syntheticTestCases\wigmosta1999/summa_fileManager-exp2.txt
converting settings_org\wrrPaperTestCases\figure01/summa_fileManager_riparianAspenBeersLaw.txt
converting settings_org\wrrPaperTestCases\figure01/summa_fileManager_riparianAspenCLM2stream.txt
converting settings_org\wrrPaperTestCases\figure01/summa_fileManager_riparianAspenNLscatter.txt
converting settings_org\wrrPaperTestCases\figure01/summa_fileManager_riparianAspenUEB2stream.txt
converting set

In [20]:
file_man = read_file_manager('summa_fileManager_columbia10.txt')
# changing the output control file name
file_man['ouput_file'] = 'oc_long.txt'
# convert initial condition file to netcdf
if not file_man['initial_cond'].endswith('.nc'):
    file_man['initial_cond'] = convert_ini_nc(os.curdir, file_man) 
# write back the file manager
write_file_manager(file_man, 'summa_fileManager_columbia10.txt')

TypeError: 'str' object is not callable

66.60795454545455